In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    StaleElementReferenceException,
    TimeoutException,
    ElementClickInterceptedException,
)
import time

# 드라이버 설정 및 URL 이동
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.set_window_size(1280, 800)
url = 'https://ev.or.kr/nportal/buySupprt/initSubsidyTargetVehicleAction.do#'
driver.get(url)

# 페이지 로드 대기
wait = WebDriverWait(driver, 10)

# 수집한 정보를 저장할 리스트
all_data = []

def crawling():
    """ 페이지에서 전기차 정보를 수집하는 함수 """
    car_name, car_person_num, max_speed, dis_per_charge, battery, sub, phone_num, maker, makerN = [], [], [], [], [], [], [], [], []

    while True:
        # 정보 수집
        try:
            car_names = driver.find_elements(By.CSS_SELECTOR, "#subPage .itemCont .scrolldelay2 a h4 p")
            car_person_nums = driver.find_elements(By.CSS_SELECTOR, "#subPage .itemCont .scrolldelay2 a dl dd:nth-child(2)")
            max_speeds = driver.find_elements(By.CSS_SELECTOR, "#subPage .itemCont .scrolldelay2 a dl dd:nth-child(3)")
            dis_per_charges = driver.find_elements(By.CSS_SELECTOR, "#subPage .itemCont .scrolldelay2 a dl dd:nth-child(4)")
            batterys = driver.find_elements(By.CSS_SELECTOR, "#subPage .itemCont .scrolldelay2 a dl dd:nth-child(5)")
            subs = driver.find_elements(By.CSS_SELECTOR, "#subPage .itemCont .scrolldelay2 a dl dd:nth-child(6)")
            phone_nums = driver.find_elements(By.CSS_SELECTOR, "#subPage .itemCont .scrolldelay2 a dl dd:nth-child(7)")
            makers = driver.find_elements(By.CSS_SELECTOR, "#subPage .itemCont .scrolldelay2 a dl dd:nth-child(8)")
            makerNs = driver.find_elements(By.CSS_SELECTOR, "#subPage .itemCont .scrolldelay2 a dl dd:nth-child(9)")

            # 리스트에 추가
            car_name.extend([i.text for i in car_names])
            car_person_num.extend([i.text for i in car_person_nums])
            max_speed.extend([i.text for i in max_speeds])
            dis_per_charge.extend([i.text for i in dis_per_charges])
            battery.extend([i.text for i in batterys])
            sub.extend([i.text for i in subs])
            phone_num.extend([i.text for i in phone_nums])
            maker.extend([i.text for i in makers])
            makerN.extend([i.text for i in makerNs])
        except Exception as e:
            print(f"데이터 수집 중 오류 발생: {e}")
            break

        # 페이지 끝까지 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        wait.until(lambda driver: driver.execute_script("return document.readyState") == "complete")  # Wait for page to load

        # 다음 페이지 버튼 클릭
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#pageingPosition > a.next.arrow')))
            driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
            next_button.click()
            wait.until(lambda driver: driver.execute_script("return document.readyState") == "complete")  # Wait for page to load
        except (StaleElementReferenceException, TimeoutException, ElementClickInterceptedException):
            print("더 이상 페이지가 없거나 클릭할 수 없습니다.")
            break

    return car_name, car_person_num, max_speed, dis_per_charge, battery, sub, phone_num, maker, makerN

def fetch_data(company_name):
    """ 지정한 회사의 정보를 크롤링하는 함수 """
    try:
        # 'schCompany' 드롭다운에서 제조사 선택
        dropdown = Select(driver.find_element(By.ID, 'schCompany'))
        dropdown.select_by_visible_text("BMW")
        time.sleep(1)  # Allow dropdown to update
        
        # 선택한 사이트로 이동하기
        driver.find_element(By.XPATH, '//*[@id="searchForm"]/div/table/tbody/tr[4]/td/button').click()
        time.sleep(3)  # 페이지 로드 대기

        # 크롤링 실행
        car_name, car_person_num, max_speed, dis_per_charge, battery, sub, phone_num, maker, makerN = crawling()

        # 수집한 정보를 저장
        for cn, cpn, ms, dpc, ba, sb, pn, ma, maN in zip(car_name, car_person_num, max_speed, dis_per_charge, battery, sub, phone_num, maker, makerN):
            if cn:  # Validate non-empty data
                all_data.append({
                    '차종': cn,
                    '승차인원': cpn,
                    '최고속도출력': ms,
                    '1회충전주행거리': dpc,
                    '배터리': ba,
                    '국고보조금': sb,
                    '판매사연락처': pn,
                    '제조사': ma,
                    '제조국가': maN
                })
    except Exception as e:
        print(f"{company_name} 정보 크롤링 중 오류 발생: {e}")

# 제조사 목록
companies = ["BMW", "기아", "현대", "폭스바겐"]  # 예시 제조사 목록

# 각 제조사에 대해 데이터 수집
for company in companies:
    fetch_data(company)

# 수집한 정보를 텍스트 파일로 저장
with open('car_data.txt', 'w', encoding='utf-8') as file:
    for data in all_data:
        file.write(f"차종: {data['차종']}\n")
        file.write(f"승차인원: {data['승차인원']}\n")
        file.write(f"최고속도출력: {data['최고속도출력']}\n")
        file.write(f"1회충전주행거리: {data['1회충전주행거리']}\n")
        file.write(f"배터리: {data['배터리']}\n")
        file.write(f"국고보조금: {data['국고보조금']}\n")
        file.write(f"판매사연락처: {data['판매사연락처']}\n")
        file.write(f"제조사: {data['제조사']}\n")
        file.write(f"제조국가: {data['제조국가']}\n")
        file.write("\n")  # 각 차량 정보 간에 빈 줄 추가

print("크롤링 완료! 데이터가 car_data.txt에 저장되었습니다.")

# 드라이버 종료
driver.quit()


BMW 정보 크롤링 중 오류 발생: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.101)
Stacktrace:
0   chromedriver                        0x000000010e218d08 chromedriver + 4996360
1   chromedriver                        0x000000010e2105ca chromedriver + 4961738
2   chromedriver                        0x000000010ddb3d10 chromedriver + 388368
3   chromedriver                        0x000000010dd8b44f chromedriver + 222287
4   chromedriver                        0x000000010de2c9fd chromedriver + 883197
5   chromedriver                        0x000000010de423f9 chromedriver + 971769
6   chromedriver                        0x000000010de24753 chromedriver + 849747
7   chromedriver                        0x000000010ddf3635 chromedriver + 648757
8   chromedriver                        0x000000010ddf3e5e chromedriver + 650846
9   chromedriver                        0x000000010e1deff0 chromedriver + 4759536
10  chromedriver      